# 3-D Segmentation Network Toy Problem Training

This notebook implements the 3D Segmentation Network to a custom toy problem. Tho objective is to segmentate volumes of different shape within a big rectangular prism. The volumes to be segmented can have different intensities.

In [1]:
# Enviroment reset
%reset  
# matplotlib plots within notebook
%matplotlib inline



# Package importing
import numpy as np
import matplotlib.pyplot as plt

from mayavi import mlab
import pylab

from collections import OrderedDict



# Custom packages
import sys
sys.path.insert(0, '../Librerias/Segm_Net_3D')

import Geometrias_3D as geo3D
import Segm_net_3D as segm3D

# Tensor Flow
import tensorflow as tf
sess = tf.InteractiveSession()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Problem variables

In [2]:
# Path to checkpoint
CHECKPOINT_PATH = "../Salidas/Segm_Net_3D_Full/Segm_Net_3D.ckpt"

# Path to tensorboard desiered output
TENSORBOARD_TRAIN_PATH = "../TensorGraph_out/Segm_Net_3D_Full/train"
TENSORBOARD_TEST_PATH = "../TensorGraph_out/Segm_Net_3D_Full/test"

# --- Network parameters

# Imput volume size
voxels_X = 128
voxels_Y = 128
voxels_Z = 32
input_size = (voxels_X,voxels_Y,voxels_Z)
# Cube fileter size of convolution layers
size_filt_fine = (3, 3, 3) 
# Cube fileter size of last output layer
size_filt_out = (1, 1, 1) 
# Down-sample cube filter size
size_filt_down = (2, 2, 2) 
# Up-mple clube filter size
sizet_up = (2, 2, 2) 
# Imput channels, only one intensity volume
input_size_1 = 1
# Output channels, one per class plus background
num_clases = 6+1
# Volume color depth in this case is a single channel (intensity image)
input_channels = 1
# Network input placeholder
ph_entry = tf.placeholder(tf.float32)
# Network state placeholder (Train or Test)
phase = tf.placeholder(tf.bool)
# Depth of the network, hoy many down-convolutions will take place before the base level
network_depth = 4
# Channels to apply at each level in the down-ward path
net_channels_down = [16, 32, 64, 128]
# Channels to apply at base level
net_channels_base = [256]
# Channels to apply at each level in the up-ward path
net_channels_up = [16, 32, 64, 128]
# Channels to apply at segmentation/output level
net_channels_segm = [1]
# Convolutional layers to apply at each level in the down-ward path
net_layers_down = [2, 2, 2, 2]
# Convolutional layers to apply at base level
net_layers_base = [2]
# Convolutional layers to apply at each level in the up-ward path
net_layers_up = [1, 2, 2, 2]

# --- Trainer parameters

# Volume mini-batch size
batch_size = 3
# Ground truth imput placeholder
ph_truth = tf.placeholder(tf.float32, [batch_size, voxels_X, voxels_Y, voxels_Z, num_clases])
# Train step size placeholder
with tf.name_scope('step_size'):
    step_size = tf.placeholder(tf.float32)
    tf.summary.scalar('step_size', step_size)
# Initial step size
step_size_ini = 1e-3

# --- Volume generation parameters

# Noise to be added
noise_max = 0.15;
# Amount of elements to be inserted in the volume
Elements_number = 10
# Margins of volume to be left unpopulated
margins = (12,12,12,12,2,2)
# Size range of objects
object_size_range = (3,12)
# Transparency range of objects
object_transp_range = (0.6,0.95)
# Gauss filtering parameter
gauss_sigma = 0.5
# Class learned (0 for multi-class)
class_learn = 0





## Network and Trainer assembly

In [3]:


# Network assembly

net_out = segm3D.Assemble_Network(ph_entry, phase, input_size, input_channels, num_clases, size_filt_fine, size_filt_out, network_depth, net_channels_down, net_layers_down, net_channels_base, net_layers_base, net_channels_up, net_layers_up, net_channels_segm)



# Trainer assembly

# Apply the dice loss, wich must be maximized
with tf.name_scope('dice_loss'):
    diff = segm3D.dice_loss(net_out, ph_truth)
with tf.name_scope('total'):
    maxim_objct = tf.reduce_mean(diff)
tf.summary.scalar('dice_loss', maxim_objct)

# Apply an ADAM optimizer wich minimizes the negated dice loss
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(step_size).minimize(-maxim_objct)
    
# Batch normalization mean and deviation updater
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)


### TensorBoard configuration

In [4]:

# Merge all the summaries and write them out to the specified path
merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(TENSORBOARD_TRAIN_PATH,
                                      sess.graph)
test_writer = tf.summary.FileWriter(TENSORBOARD_TEST_PATH)

# Train the model and save it in the end
model_saver = tf.train.Saver()

## Training

In [5]:

# Variable initialization
sess.run(tf.global_variables_initializer())

step_size_act = step_size_ini
# Train Loop
for i in range(0, 20000):
    # Random mini batch generation
    (batch_in, batch_label) = geo3D.fun_gen_batch(batch_size,
                                               [voxels_X, voxels_Y, voxels_Z], 
                                               noise_max, 
                                               Elements_number, 
                                               margins, 
                                               object_size_range, 
                                               object_transp_range, 
                                               gauss_sigma,
                                               class_learn)
    # Cast ground truth to int32
    batch_label = batch_label.astype(int)

    # Every 100 iterations we test the model
    if i%100 == 0:
        summary, train_accuracy = sess.run([merged, maxim_objct], feed_dict={ph_entry:batch_in, ph_truth: batch_label, step_size: step_size_act, phase: False})
        test_writer.add_summary(summary, i)
        # Every 100 iterations we print the diceloss
        if i%1000 == 0:
            print("step %d, diceloss: %g"%(i, train_accuracy))
        
        
    # Train
    summary, _, _ = sess.run([merged, train_step, extra_update_ops], feed_dict={ph_entry: batch_in, ph_truth: batch_label, step_size: step_size_act, phase: True})
    train_writer.add_summary(summary, i)
    
    # Update train step
    step_size_act = step_size_ini / np.sqrt(i+1)
    

# Save the trained network
model_saver.save(sess, CHECKPOINT_PATH)


step 0, diceloss: 0.0592543
step 1000, diceloss: 0.620119
step 2000, diceloss: 0.678825
step 3000, diceloss: 0.711138
step 4000, diceloss: 0.782142
step 5000, diceloss: 0.812811
step 6000, diceloss: 0.839637
step 7000, diceloss: 0.858026
step 8000, diceloss: 0.873125
step 9000, diceloss: 0.889265
step 10000, diceloss: 0.89984
step 11000, diceloss: 0.912346
step 12000, diceloss: 0.906259
step 13000, diceloss: 0.931157
step 14000, diceloss: 0.935945
step 15000, diceloss: 0.941711
step 16000, diceloss: 0.947603
step 17000, diceloss: 0.951023
step 18000, diceloss: 0.953619
step 19000, diceloss: 0.955359


'../Salidas/test/Segm_Net_3D.ckpt'